In [1]:
import pandas as pd
import panel as pn
import numpy as np
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set

pn.extension('tabulator')

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 6, 21)

### Set today = last closed business day

In [2]:
#today = today - timedelta(days=1)
today

datetime.date(2022, 6, 21)

### Restart and Run All Cells

In [3]:
sqlUpd = """
UPDATE buy B
SET dividend =
(SELECT DIVIDEND FROM dividend D
WHERE B.name = D.name)"""
rp = const.execute(sqlUpd)
rp.rowcount

27

In [4]:
cols = 'name period shares mkt_price cost_amt base_amt pct'.split()
colt = 'name shares unit_cost cost_amt price market_amt mkt_pct div_rnk amt_rnk div_amt'.split()
colu = 'name shares unit_cost cost_amt dividend div_amt cost_pct period'.split()
colv = 'name shares unit_cost cost_amt mkt_price market_amt dividend div_amt cost_pct mkt_pct profit pft_pct'.split()
colw = 'name cost_amt market_amt div_amt cost_pct mkt_pct pft_pct'.split()
colx = 'name shares unit_cost cost_amt mkt_price market_amt dividend div_amt cost_pct mkt_pct profit pft_pct period'.split()

In [5]:
format_dict = {
    'shares':'{:,}',  
    'mkt_price':'{:.2f}','unit_cost':'{:.2f}',
    'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','market_amt':'{:,.2f}','profit':'{:,.2f}','base_amt':'{:,.2f}',
    'dividend':'{:.4f}',    
    'pct':'{:,.2f}%','cost_pct':'{:,.2f}%','mkt_pct':'{:,.2f}%','pft_pct':'{:,.2f}%',
}

### Process portfolio (table buy in mysql stock)

In [6]:
sql = """
SELECT name, volbuy, price AS unit_cost, dividend, period
FROM buy
WHERE active = 1"""
print(sql)


SELECT name, volbuy, price AS unit_cost, dividend, period
FROM buy
WHERE active = 1


In [7]:
buy = pd.read_sql(sql, const)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy['cost_amt'] = round(buy['shares'] * buy['unit_cost'], 2)
buy['cost_pct'] = round(buy['dividend'] / buy['unit_cost'] * 100, 2)
buy['div_amt'] = round(buy['shares'] * buy['dividend'], 2)
buy[colu].sort_values('cost_pct',ascending=False).head().style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
5,TMT,"48,000",9.90,"475,200.00",1.3000,"62,400.00",13.13%,2
24,RCL,"12,000",47.25,"567,000.00",6.0000,"72,000.00",12.70%,3
12,ASP,"30,000",3.80,"114,000.00",0.4000,"12,000.00",10.53%,2
11,JASIF,"110,000",10.10,"1,111,000.00",0.9400,"103,400.00",9.31%,2
3,MCS,"75,000",15.40,"1,155,000.00",1.2700,"95,250.00",8.25%,2


In [8]:
file_name = 'hi-dividend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(output_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(data_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(box_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(one_file, index=False)

### Start of Period Calculation

In [9]:
sql = '''
SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B JOIN price P
ON B.name = P.name
WHERE P.date = "%s"
AND active = 1'''
sql = sql % today 
print(sql)


SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B JOIN price P
ON B.name = P.name
WHERE P.date = "2022-06-21"
AND active = 1


In [10]:
df = pd.read_sql(sql, const)
df.rename(columns={'volbuy':'shares'},inplace=True)
df['shares'] = df.shares.astype(int)
df['cost_amt'] = round(df['shares'] * df['unit_cost'],2)
df['market_amt'] = round(df['shares'] * df['mkt_price'],2)
df['div_amt'] = round(df['shares'] * df['dividend'],2)
df['cost_pct'] = round(df['dividend'] / df['unit_cost'] * 100,2)
df['mkt_pct'] = round(df['dividend'] / df['mkt_price'] * 100,2)
df['profit'] = round((df['mkt_price'] - df['unit_cost']) * df['shares'],2)
df['pft_pct'] = round(df['profit'] / df['cost_amt'] * 100,2)
df[colv].sort_values(['pft_pct'],ascending=[False]).head().style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
11,JASIF,"110,000",10.10,"1,111,000.00",10.50,"1,155,000.00",0.9400,"103,400.00",9.31%,8.95%,"44,000.00",3.96%
25,TTB,"100,000",1.26,"126,000.00",1.26,"126,000.00",0.0380,"3,800.00",3.02%,3.02%,0.00,0.00%
26,SIS,"3,600",28.50,"102,600.00",27.25,"98,100.00",1.2000,"4,320.00",4.21%,4.40%,"-4,500.00",-4.39%
1,BDMS,"4,500",26.00,"117,000.00",24.80,"111,600.00",0.4500,"2,025.00",1.73%,1.81%,"-5,400.00",-4.62%
21,DCC,"60,000",2.96,"177,600.00",2.76,"165,600.00",0.1900,"11,400.00",6.42%,6.88%,"-12,000.00",-6.76%


In [11]:
df.nlargest(5, 'mkt_pct')[colw].style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct,pft_pct
5,TMT,"475,200.00","424,800.00","62,400.00",13.13%,14.69%,-10.61%
24,RCL,"567,000.00","507,000.00","72,000.00",12.70%,14.20%,-10.58%
12,ASP,"114,000.00","94,800.00","12,000.00",10.53%,12.66%,-16.84%
3,MCS,"1,155,000.00","900,000.00","95,250.00",8.25%,10.58%,-22.08%
15,WHAIR,"358,000.00","298,000.00","27,080.00",7.56%,9.09%,-16.76%


In [12]:
df.nsmallest(5, 'mkt_pct')[colw].style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct,pft_pct
10,DOHOME,"145,800.00","120,240.00",0.00,0.00%,0.00%,-17.53%
1,BDMS,"117,000.00","111,600.00","2,025.00",1.73%,1.81%,-4.62%
6,MAKRO,"303,000.00","258,750.00","5,400.00",1.78%,2.09%,-14.60%
2,KCE,"1,076,250.00","945,000.00","24,000.00",2.23%,2.54%,-12.20%
23,SCC,"358,200.00","326,700.00","8,999.91",2.51%,2.75%,-8.79%


In [13]:
file_name = 'buy-div-price.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(output_file)
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(data_file)
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(box_file)

### Short term stocks

In [14]:
short_term = df.period == '4'
df.loc[short_term][colv].sort_values(['pft_pct'],ascending=[False]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
23,SCC,900,398.00,"358,200.00",363.00,"326,700.00",9.9999,"8,999.91",2.51%,2.75%,"-31,500.00",-8.79%
8,BCH,"21,000",20.90,"438,900.00",18.10,"380,100.00",1.2000,"25,200.00",5.74%,6.63%,"-58,800.00",-13.40%
6,MAKRO,"7,500",40.40,"303,000.00",34.50,"258,750.00",0.7200,"5,400.00",1.78%,2.09%,"-44,250.00",-14.60%
19,NER,"27,000",7.45,"201,150.00",6.30,"170,100.00",0.4300,"11,610.00",5.77%,6.83%,"-31,050.00",-15.44%
10,DOHOME,"7,200",20.25,"145,800.00",16.70,"120,240.00",0.0000,0.00,0.00%,0.00%,"-25,560.00",-17.53%


In [15]:
p4cost = df.loc[short_term].cost_amt.sum()
p4profit = df.loc[short_term].profit.sum()
p4pct = round(p4profit/p4cost*100, 2)
p4cost, p4profit, p4pct, df.loc[short_term].shape[0]

(1447050.0, -191160.0, -13.21, 5)

### Long term stocks

In [16]:
long_term = df.period == '3'
df[long_term].sort_values(['pft_pct'],ascending=[False]).shape

(8, 13)

In [17]:
df[long_term].nlargest(5, 'pft_pct')[['name','cost_amt','market_amt','profit','pft_pct']].style.format(format_dict)

,name,cost_amt,market_amt,profit,pft_pct
25,TTB,"126,000.00","126,000.00",0.00,0.00%
26,SIS,"102,600.00","98,100.00","-4,500.00",-4.39%
1,BDMS,"117,000.00","111,600.00","-5,400.00",-4.62%
16,ORI,"495,000.00","454,500.00","-40,500.00",-8.18%
24,RCL,"567,000.00","507,000.00","-60,000.00",-10.58%


In [18]:
p3cost = df.loc[long_term].cost_amt.sum()
p3profit = df.loc[long_term].profit.sum()
p3pct = round(p3profit/p3cost*100, 2)
p3cost, p3profit, p3pct,df.loc[long_term].shape[0]

(2758350.0, -271950.0, -9.86, 8)

### High dividend stocks

In [19]:
hi_dividend = df.period == '2'
df.loc[hi_dividend][colv].sort_values(['mkt_pct'],ascending=[False]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
5,TMT,"48,000",9.90,"475,200.00",8.85,"424,800.00",1.3000,"62,400.00",13.13%,14.69%,"-50,400.00",-10.61%
12,ASP,"30,000",3.80,"114,000.00",3.16,"94,800.00",0.4000,"12,000.00",10.53%,12.66%,"-19,200.00",-16.84%
3,MCS,"75,000",15.40,"1,155,000.00",12.00,"900,000.00",1.2700,"95,250.00",8.25%,10.58%,"-255,000.00",-22.08%
15,WHAIR,"40,000",8.95,"358,000.00",7.45,"298,000.00",0.6770,"27,080.00",7.56%,9.09%,"-60,000.00",-16.76%
11,JASIF,"110,000",10.10,"1,111,000.00",10.50,"1,155,000.00",0.9400,"103,400.00",9.31%,8.95%,"44,000.00",3.96%
9,SENA,"105,000",4.48,"470,400.00",4.06,"426,300.00",0.3405,"35,752.50",7.60%,8.39%,"-44,100.00",-9.38%
13,TISCO,"2,000",97.50,"195,000.00",89.00,"178,000.00",7.1500,"14,300.00",7.33%,8.03%,"-17,000.00",-8.72%
7,WHART,"30,000",11.70,"351,000.00",9.70,"291,000.00",0.7578,"22,734.00",6.48%,7.81%,"-60,000.00",-17.09%
4,DIF,"40,000",14.70,"588,000.00",13.40,"536,000.00",1.0430,"41,720.00",7.10%,7.78%,"-52,000.00",-8.84%
21,DCC,"60,000",2.96,"177,600.00",2.76,"165,600.00",0.1900,"11,400.00",6.42%,6.88%,"-12,000.00",-6.76%


In [20]:
df[hi_dividend].nlargest(5, 'mkt_pct')[['name','cost_amt','market_amt','div_amt','cost_pct','mkt_pct']]\
.style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct
5,TMT,"475,200.00","424,800.00","62,400.00",13.13%,14.69%
12,ASP,"114,000.00","94,800.00","12,000.00",10.53%,12.66%
3,MCS,"1,155,000.00","900,000.00","95,250.00",8.25%,10.58%
15,WHAIR,"358,000.00","298,000.00","27,080.00",7.56%,9.09%
11,JASIF,"1,111,000.00","1,155,000.00","103,400.00",9.31%,8.95%


In [21]:
p2profit = df.loc[hi_dividend].profit.sum()
p2cost = df.loc[hi_dividend].cost_amt.sum()
p2dividend = df.loc[hi_dividend].div_amt.sum()
p2yield = round(p2profit/p2cost*100,2)
p2cost, p2profit, p2yield, p2dividend, df.loc[hi_dividend].shape[0]

(5200400.0, -551500.0, -10.6, 438036.38, 11)

### Disposal stocks

In [22]:
disposal = df.period == '1'
df.loc[disposal][colv].sort_values(['pft_pct'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
0,STA,"22,500",33.75,"759,375.00",21.70,"488,250.00",1.9000,"42,750.00",5.63%,8.76%,"-271,125.00",-35.70%
22,SYNEX,"18,000",28.00,"504,000.00",18.40,"331,200.00",0.7000,"12,600.00",2.50%,3.80%,"-172,800.00",-34.29%
14,PTTGC,"12,000",58.75,"705,000.00",44.75,"537,000.00",3.7500,"45,000.00",6.38%,8.38%,"-168,000.00",-23.83%


In [23]:
p1cost = df.loc[disposal].cost_amt.sum()
p1profit = df.loc[disposal].profit.sum()
p1pct = round(p1profit/p1cost*100,2)
p1cost, p1profit, p1pct,df.loc[disposal].shape[0]

(1968375.0, -611925.0, -31.09, 3)